In [1]:
import pandas as pd

In [2]:
# Chargement des fichiers Excel
web = pd.read_excel("data/Fichier_web.xlsx")
erp = pd.read_excel("data/Fichier_erp.xlsx")
liaison = pd.read_excel("data/fichier_liaison.xlsx")

c:\Users\Dev\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\Dev\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\Dev\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
# Nettoyage : suppression des lignes vides
web = web.dropna(subset=["sku", "post_title"])
erp = erp.dropna(subset=["product_id", "price"])
liaison = liaison.dropna()

# Supprimer les doublons
web = web.drop_duplicates(subset="sku")
erp = erp.drop_duplicates(subset="product_id")
liaison = liaison.drop_duplicates()

In [5]:
# On convertit les types si nécessaire
web["sku"] = web["sku"].astype(str)
liaison["id_web"] = liaison["id_web"].astype(str)

# Jointure liaison + web
df = pd.merge(liaison, web, left_on="id_web", right_on="sku", how="inner")

# Puis jointure avec ERP
df = pd.merge(df, erp, on="product_id", how="inner")


In [6]:
# Ajout d'une colonne chiffre d'affaires (CA)
df["CA"] = df["price"] * df["stock_quantity"]

# CA total
ca_total = df["CA"].sum()

# CA par produit
ca_par_produit = df.groupby("post_title")["CA"].sum().reset_index()


In [7]:
from scipy.stats import zscore

# Calcul du z-score sur les prix
df["z_score_price"] = zscore(df["price"])

# Classification
df["type_vin"] = df["z_score_price"].apply(lambda z: "premium" if z > 1 else "ordinaire")

# Séparation
vins_premium = df[df["type_vin"] == "premium"]
vins_ordinaires = df[df["type_vin"] == "ordinaire"]


In [8]:
# Sauvegarde en Excel
with pd.ExcelWriter("rapport_vins.xlsx") as writer:
    ca_par_produit.to_excel(writer, sheet_name="CA_par_produit", index=False)
    pd.DataFrame([{"CA_total": ca_total}]).to_excel(writer, sheet_name="CA_total", index=False)
    vins_premium.to_excel(writer, sheet_name="Vins_premium", index=False)
    vins_ordinaires.to_excel(writer, sheet_name="Vins_ordinaires", index=False)


In [9]:
assert not web.isnull().any().any(), "Valeurs manquantes dans 'web'"
assert not erp.isnull().any().any(), "Valeurs manquantes dans 'erp'"
assert df["CA"].min() >= 0, "CA négatif détecté"
assert "premium" in df["type_vin"].values, "Aucun vin premium détecté"


AssertionError: Valeurs manquantes dans 'web'